In [1]:
# RetinaFace 사용을 위한 필수 라이브러리를 설치합니다.
!pip install opencv-python matplotlib

In [2]:
# Pytorch_Retinaface 경로를 설정합니다
import sys
sys.path.append(r'c:\ai_project01\Pytorch_Retinaface')

In [3]:
# 필요한 라이브러리들을 불러옵니다.
import torch
import cv2
import numpy as np

from models.retinaface import RetinaFace
from data import cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
from utils.box_utils import decode

In [5]:
# GPU 사용 여부를 검색합니다
torch.cuda.is_available()

True

In [6]:
device = "cuda"

In [7]:
cfg = cfg_re50
net = RetinaFace(cfg=cfg, phase='test').to(device)

C:\ProgramData\anaconda3\envs\yolo_env01\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\envs\yolo_env01\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\User/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 98.2MB/s]


In [8]:
pretrained_path = r'c:\ai_project01\Pytorch_Retinaface\weights\Resnet50_Final.pth'

In [9]:
state_dict = torch.load(pretrained_path, map_location=device)

C:\Users\User\AppData\Local\Temp\ipykernel_8952\4222367130.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_path, map_location=device)


In [10]:
new_state_dict = {}

for k, v in state_dict.items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v

In [11]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [12]:
net.eval()

RetinaFace(
  (body): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seque

In [17]:
image_path = r'c:\ai_project01\mask_images\no_mask\no_mask_00000.png'

img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)

img = np.float32(img_raw)

In [18]:
im_height, im_width, _ = img.shape

scale = torch.Tensor([im_width, im_height, im_width, im_height]).to(device)

In [19]:
img -= (104, 117, 123)

In [20]:
img = img.transpose(2, 0, 1)

In [21]:
img = torch.from_numpy(img).unsqueeze(0).to(device)

In [22]:
loc, conf, landms = net(img)

In [23]:
priorbox = PriorBox(cfg, image_size=(im_height, im_width))

In [24]:
priors = priorbox.forward().to(device)

In [25]:
prior_data = priors.data

In [26]:
boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])

In [27]:
boxes = boxes * scale

In [29]:
scores = conf.squeeze(0).data.cpu().numpy()[:,1]

In [30]:
confidence_threshold = 0.5

top_indices = np.where(scores > confidence_threshold)[0]

In [31]:
boxes = boxes[top_indices]

scores = scores[top_indices]

In [32]:
dets = np.hstack((boxes.cpu().numpy(), scores[:, np.newaxis])).astype(np.float32, copy=False)

In [33]:
keep = py_cpu_nms(dets, 0.3)

In [34]:
dets = dets[keep, :]

In [38]:
for b in dets:
    if b[4] < confidence_threshold:
        continue
    text = "{:.2f}".format(b[4])
    
    b = list(map(int, b))

    cv2.rectangle(img_raw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 2)

    cx, cy = b[0], b[1] + 12
    cv2.putText(img_raw, text, (cx, cy), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

In [ ]:
cv2.imshow('RetinaFace PyTorch Detection', img_raw)

cv2.waitKey(0)

cv2.destroyAllWindows()